In [1]:
import SimpleITK as sitk 
import numpy as np 
import matplotlib.pyplot as plt 
from library_dicom.dicom_processor.tools.preprocessing import *
import json 
import csv 
from sklearn.model_selection import train_test_split 
import tensorflow as tf 
from classification.Prep_CSV import Prep_CSV
from classification.Preprocessing import Preprocessing 
from classification.resnet50 import *

In [ ]:
json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/result.json'
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92'
objet = Prep_CSV(json_path)
objet.result_csv(nifti_directory)
print(objet.csv_result_path)

prep_objet = Preprocessing(objet.csv_result_path)
X, y = prep_objet.normalize_encoding_dataset()

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, test_size = 0.15)

print("size of X_train : ", X_train.shape)
print("size of y_train : ",y_train.shape)
print("")
print("size of X_test : ", X_test.shape)
print("size of y_test : ",y_test.shape)
#print("")
#print("size of X_val : ", X_val.shape)
#print("size of y_val : ",y_val.shape)


#model 
model = ResNet50(input_shape=(1024, 256, 1))
model.summary()
#compile 
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4) #param
model.compile(optimizer = optimizer, 
        loss={'left_arm' : 'sparse_categorical_crossentropy', 
            'right_arm' : 'sparse_categorical_crossentropy', 
             'head' : 'sparse_categorical_crossentropy', 
             'leg' : 'sparse_categorical_crossentropy'}, 
        loss_weights ={'left_arm': 0.25, 'right_arm' : 0.25, 
                        'head' : 0.25, 
                        'leg': 0.25}, 
        metrics = {'left_arm': ['accuracy'], #'SparseCategoricalCrossentropy'
                    'right_arm' : ['accuracy'], 
                    'head' : ['accuracy'], 
                    'leg':['accuracy']}) #a voir pour loss

#log_dir = '/home/deeplearning/Dicom-To-CNN/Classification/Resnet50/logs'
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='epoch', write_graph=True, write_images=True)


In [ ]:
history = model.fit(X_train, {'head': y_train[:,0], 
                                    'leg': y_train[:,1],
                                    'right_arm' : y_train[:,2],
                                    'left_arm' : y_train[:,3] ,
                                    }, 
                                    
                        epochs = 10, 
                        batch_size = 50, 
                        verbose = 1, 
                        validation_split= 0.15)
                        #validation_data = (X_val, {'head': y_val[:,0], 
                                    #'leg': y_val[:,1],
                                    #'right_arm' : y_val[:,2],
                                    #'left_arm' : y_val[:,3] ,
                                    #}))
                        #callbacks=[tensorboard_callback])

#print(y_val)
#predictions = model.predict(X_val[0:10])
#print("Predictions done !")
#print(predictions)
#from classification.resnet50 import predictions_decoding 
#predi = predictions_decoding(predictions)
#print(predi)


#save 
model.save('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/Classification/Resnet50/resnet50', save_format='h5')

In [11]:
pred = [np.array([[0.35805932, 0.325679  , 0.3162617 ],
       [0.3583647 , 0.3258384 , 0.3157969 ],
       [0.35832056, 0.32527184, 0.31640765],
       [0.35825935, 0.32591996, 0.31582072],
       [0.35852462, 0.3256994 , 0.31577596],
       [0.35781434, 0.3257167 , 0.31646898],
       [0.35850057, 0.3255967 , 0.31590277],
       [0.35806704, 0.32569864, 0.31623432],
       [0.35794508, 0.32569015, 0.3163648 ],
       [0.35804483, 0.32574567, 0.3162095 ]]), np.array([[0.35203746, 0.3414531 , 0.30650944],
       [0.35249195, 0.34101707, 0.306491  ],
       [0.35207516, 0.34141713, 0.3065077 ],
       [0.35236207, 0.3414479 , 0.30619007],
       [0.35234544, 0.34109735, 0.30655718],
       [0.35247347, 0.34153646, 0.30599004],
       [0.35245287, 0.34099725, 0.30654997],
       [0.35227928, 0.34146503, 0.30625573],
       [0.35257983, 0.3415341 , 0.30588606],
       [0.3521983 , 0.34119767, 0.306604  ]]), np.array([[0.49849725, 0.50150275],
       [0.49877256, 0.5012274 ],
       [0.49910057, 0.50089943],
       [0.4987126 , 0.5012874 ],
       [0.49852982, 0.5014702 ],
       [0.4990961 , 0.50090384],
       [0.49894577, 0.50105417],
       [0.4986342 , 0.5013658 ],
       [0.49890473, 0.50109524],
       [0.4987169 , 0.5012831 ]]), np.array([[0.5195359 , 0.4804641 ],
       [0.51906794, 0.4809321 ],
       [0.51953894, 0.480461  ],
       [0.51931673, 0.48068327],
       [0.51934695, 0.48065302],
       [0.5194422 , 0.48055786],
       [0.519488  , 0.480512  ],
       [0.5195441 , 0.4804559 ],
       [0.51943207, 0.48056793],
       [0.51944226, 0.4805577 ]])]


In [38]:
pred[0].shape[0]
predi = []
for i in range(0, 1) : 
    subliste = []
    for j in range(0,4):
        liste = pred[j][i].tolist()
        print(liste)
        maxi = np.max(liste)
        index = liste.index(maxi)
        print(index)
        if j == 0 : #head
            if index == 0 : 
                subliste.append("Vertex")
            if index == 1 : 
                subliste.append("Eyes")
            if index == 2 : 
                subliste.append("Mouth")
        if j == 1 : #leg
            if index == 0 : 
                subliste.append("Hips")
            if index == 1 : 
                subliste.append("Knee")
            if index == 2 : 
                subliste.append("Foot")
    predi.append(subliste)

[0.35805932, 0.325679, 0.3162617]
0
[0.35203746, 0.3414531, 0.30650944]
0
[0.49849725, 0.50150275]
1
[0.5195359, 0.4804641]
0


In [39]:
predi

[['Vertex', 'Hips']]